In [1]:
%load_ext autoreload
%autoreload 2

%config InlineBackend.figure_format = "retina"

In [2]:
from kpm_linreg import *
from bong.src import bbb, blr, bog, bong, experiment_utils
from bong.util import run_rebayes_algorithm, tune_init_hyperparam

print(AGENT_TYPES)


['fg-bong', 'fg-l-bong', 'fg-rep-bong', 'fg-rep-l-bong', 'fg-blr', 'fg-bog', 'fg-rep-bog', 'fg-bbb', 'fg-rep-bbb']


In [3]:
#python experiments/s01_linreg.py --agents fg-bong fg-l-bong --num_samples 10 100

parser = argparse.ArgumentParser()
    
  # Data parameters
parser.add_argument("--num_examples", type=int, default=500)
parser.add_argument("--param_dim", type=int, default=10)
parser.add_argument("--key", type=int, default=0)
parser.add_argument("--emission_noise", type=float, default=1.0)

# Model parameters
parser.add_argument("--agents", type=str, nargs="+",
                    default=["fg-bong", "fg-blr", "fg-bog", "fg-bbb"], choices=AGENT_TYPES)
parser.add_argument("--num_samples", type=int, nargs="+", 
                    default=[100])
parser.add_argument("--learning_rate", type=int, nargs="+", 
                    default=[0.005, 0.01, 0.05])
parser.add_argument("--tune_learning_rate", type=bool, default=False)

#https://stackoverflow.com/questions/30656777/how-to-call-module-written-with-argparse-in-ipython-notebook
#args = parser.parse_args()
args = parser.parse_args(args=[])
print(args)

#main(args)


In [8]:
data, subkey = make_data(args)
init_kwargs, callback = init(args, data)
prior, post = compute_prior_post(args, data)
 

agent = bog.fg_bog(**init_kwargs,
                learning_rate=0.1,
                num_samples=100,
                num_iter = 100,
)
print(agent)


TypeError: fg_bog.__new__() got an unexpected keyword argument 'num_iter'